In [9]:
import numpy as np
import pandas as pd
import json

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.python.keras.optimizers import Adam

In [2]:
dataframe = pd.read_csv('yorumlar.csv', sep = "\t")
target = dataframe['Puan'].values.tolist()
data = dataframe['Yorum'].values.tolist()

In [3]:
dataframe

,Puan,Yorum
0,1.0,Hızlı ve özenli kargo bu fiyata gayet iyi bir ...
1,1.0,"Harika bir kitqp,şiddetle tavsiye??"
2,1.0,Şahane bir plak ! Mermer baskısı o kadar şık o...
3,1.0,İnanılmaz keyifli bir oyun. Hem heyecan hem ke...
4,1.0,Alısverışten çok memnun kaldım eksıksız. Ç sor...
...,...,...
28978,1.0,yer kısıtı olanlar için çok kullanışlı bir ürü...
28979,0.0,içi boş bir kitap her sayfa birbirinden alakas...
28980,1.0,uygun fiyat
28981,1.0,Ürün elime ulaştı. Henüz okumadım ama firma ge...


In [5]:
len(data)

28983

In [6]:
cutoff = int(len(data) * 0.85)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [7]:
len(x_train)

24635

In [8]:
x_train[245]

'gitar kursuna yeni başladım. ne almalıyım diye bakınırken Hepsiburada da bu ürünü gördüm. fiyatı ve hediyeleri ile birlikte çok iyi fiyat. siyah olmasıda benim için ayrı güzel.'

In [10]:
with open('tokenizer.json') as json_dosyasi:
    json_tokenizer = json.load(json_dosyasi)

In [32]:
json_tokenizer['yakışmamış']

KeyError: 'yakışmamış'

In [13]:
def tokenlestir(yorumListesi):
    y_yorumlar = []
    for yorum in yorumListesi:
        y_yorum = []
        for kelime in yorum.split():
            if (len(y_yorum) < 50 and kelime in json_tokenizer):
                y_yorum.append(json_tokenizer[kelime])
                
        if (len(y_yorum) < 50):
            sifirlar = list(np.zeros(50 - len(y_yorum), dtype = int))
            y_yorum = sifirlar + y_yorum
            
        y_yorumlar.append(y_yorum)
    return np.array(y_yorumlar, dtype = np.dtype(np.int32))

In [14]:
egitim_kume = tokenlestir(x_train)

In [15]:
egitim_kume[245]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,  152, 3872,   76,
         66,   86,   16,    6,   67,  127,    3, 4443,   33,  285,    2,
         11,  919, 3228,  150,    7,  247], dtype=int32)

In [17]:
test_kume = tokenlestir(x_test)

In [21]:
model = Sequential()

embedding_size = 50

model.add(Embedding(input_dim=10001,
                    output_dim=embedding_size,
                    input_length=50,
                    name='embedding_layer'))

model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=1e-3)

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 50, 50)            500050    
_________________________________________________________________
gru_3 (GRU)                  (None, 50, 16)            3216      
_________________________________________________________________
gru_4 (GRU)                  (None, 50, 8)             600       
_________________________________________________________________
gru_5 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 504,027
Trainable params: 504,027
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(egitim_kume, y_train, epochs = 5, batch_size = 256)

Epoch 1/5
24635/24635 [==============================] - 12s 477us/sample - loss: 0.3353 - acc: 0.9275
Epoch 2/5
24635/24635 [==============================] - 10s 409us/sample - loss: 0.2036 - acc: 0.9354
Epoch 3/5
24635/24635 [==============================] - 10s 408us/sample - loss: 0.1481 - acc: 0.9523
Epoch 4/5
24635/24635 [==============================] - 10s 403us/sample - loss: 0.1087 - acc: 0.9673
Epoch 5/5
24635/24635 [==============================] - 10s 409us/sample - loss: 0.0839 - acc: 0.9776


In [23]:
model.evaluate(test_kume, y_test)

4348/4348 [==============================] - 2s 571us/sample - loss: 0.2093 - acc: 0.9322


[0.20926746494362083, 0.9321527]

In [24]:
model.save('28ekim14_08.h5')

In [26]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
yazilar = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [27]:
deneme_kume = tokenlestir(yazilar)

In [29]:
deneme_kume[3]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 59],
      dtype=int32)

In [30]:
sonuc = model.predict(deneme_kume)

In [31]:
sonuc

array([[0.9857707 ],
       [0.9853661 ],
       [0.17877862],
       [0.9846419 ],
       [0.4203341 ],
       [0.51625293],
       [0.9753051 ],
       [0.5689444 ],
       [0.9857189 ],
       [0.27126715]], dtype=float32)